# Testing Notebook

This notebook provides some test code/ experimentation for the implementation of centrality analysis from the paper *On the Concept of Depth from Functional Data*

Warning: the work here is very rough.

The following function produces all subsequences of length `l` from the list `s`. 

In [4]:
from itertools import combinations 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from statdepth import FunctionalDepth
from statdepth.testing import generate_noisy_univariate

We now test the standard definition of $V(x_{i_1}, ... ,x_{i_j})$ containment in $\mathbb{R}^2$ 

And define a fake dataset to do so

In [5]:
df = generate_noisy_univariate()    
bd = FunctionalDepth([df], J=2, quiet=False)

100%|██████████| 20/20 [00:02<00:00,  8.51it/s]


In [6]:
bd.plot_deepest(n=2)

Now let's view the matrix of homogeneity coefficients from each species

In [11]:
hc = pd.read_csv('../../experiment-testing/data/homogeneity_coefficients_P2.csv').set_index('Unnamed: 0')

In [12]:
hc

,Pan_troglodytes,Pongo_abelii,Homo_sapiens,Macaca_mulatta
Unnamed: 0,,,,
Pan_troglodytes,0.000000,0.407529,0.091385,0.121111
Pongo_abelii,0.407595,0.000000,0.089685,0.010954
Homo_sapiens,0.401725,0.040257,0.000000,0.041816
Macaca_mulatta,0.405904,0.000000,0.069196,0.000000


In [19]:
hc.index.name = ''

In [20]:
hc

,Pan_troglodytes,Pongo_abelii,Homo_sapiens,Macaca_mulatta
,,,,
Pan_troglodytes,0.000000,0.407529,0.091385,0.121111
Pongo_abelii,0.407595,0.000000,0.089685,0.010954
Homo_sapiens,0.401725,0.040257,0.000000,0.041816
Macaca_mulatta,0.405904,0.000000,0.069196,0.000000
